In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#veri yükleme
veriler = pd.read_csv('odev_tenis.csv')
print(veriler)

     outlook  temperature  humidity  windy play
0      sunny           85        85  False   no
1      sunny           80        90   True   no
2   overcast           83        86  False  yes
3      rainy           70        96  False  yes
4      rainy           68        80  False  yes
5      rainy           65        70   True   no
6   overcast           64        65   True  yes
7      sunny           72        95  False   no
8      sunny           69        70  False  yes
9      rainy           75        80  False  yes
10     sunny           75        70   True  yes
11  overcast           72        90   True  yes
12  overcast           81        75  False  yes
13     rainy           71        91   True   no


In [14]:
#tüm verileri aynı anda encoding etme
from sklearn import preprocessing
veriler2 = veriler.apply(preprocessing.LabelEncoder().fit_transform) 
print(veriler2)

    outlook  temperature  humidity  windy  play
0         2           11         4      0     0
1         2            8         6      1     0
2         0           10         5      0     1
3         1            4         9      0     1
4         1            2         3      0     1
5         1            1         1      1     0
6         0            0         0      1     1
7         2            6         8      0     0
8         2            3         1      0     1
9         1            7         3      0     1
10        2            7         1      1     1
11        0            6         6      1     1
12        0            9         2      0     1
13        1            5         7      1     0


In [33]:
c = veriler2.iloc[:,:1]
ohe = preprocessing.OneHotEncoder()
c = ohe.fit_transform(c).toarray()
havadurumu = pd.DataFrame(data=c, index= range(14), columns= ['o', 'r', 's'])
sonveriler = pd.concat([havadurumu, veriler.iloc[:,1:3]], axis=1)
sonveriler = pd.concat([veriler2.iloc[:,-2:],sonveriler], axis=1)
print(sonveriler)

    windy  play    o    r    s  temperature  humidity
0       0     0  0.0  0.0  1.0           85        85
1       1     0  0.0  0.0  1.0           80        90
2       0     1  1.0  0.0  0.0           83        86
3       0     1  0.0  1.0  0.0           70        96
4       0     1  0.0  1.0  0.0           68        80
5       1     0  0.0  1.0  0.0           65        70
6       1     1  1.0  0.0  0.0           64        65
7       0     0  0.0  0.0  1.0           72        95
8       0     1  0.0  0.0  1.0           69        70
9       0     1  0.0  1.0  0.0           75        80
10      1     1  0.0  0.0  1.0           75        70
11      1     1  1.0  0.0  0.0           72        90
12      0     1  1.0  0.0  0.0           81        75
13      1     0  0.0  1.0  0.0           71        91


In [34]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test =  train_test_split(sonveriler.iloc[:,:-1], sonveriler.iloc[:,-1:], test_size=0.33, random_state=0)

In [30]:
x_train, x_test, y_train, y_test

(      o    r    s  temperature
 13  0.0  1.0  0.0           71
 9   0.0  1.0  0.0           75
 1   0.0  0.0  1.0           80
 7   0.0  0.0  1.0           72
 10  0.0  0.0  1.0           75
 3   0.0  1.0  0.0           70
 0   0.0  0.0  1.0           85
 5   0.0  1.0  0.0           65
 12  1.0  0.0  0.0           81,
       o    r    s  temperature
 8   0.0  0.0  1.0           69
 6   1.0  0.0  0.0           64
 4   0.0  1.0  0.0           68
 11  1.0  0.0  0.0           72
 2   1.0  0.0  0.0           83,
     humidity
 13        91
 9         80
 1         90
 7         95
 10        70
 3         96
 0         85
 5         70
 12        75,
     humidity
 8         70
 6         65
 4         80
 11        90
 2         86)

In [35]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(x_train,y_train)
y_pred = regressor.predict(x_test)
print(y_pred)

[[84.45365573]
 [63.93839954]
 [85.76050662]
 [64.21013241]
 [75.06793322]]


In [39]:
#p-value kullanarak değişkenlerin önem derecesine göre işleme sokmak
import statsmodels.api as sm
# beta 0 değerlerini ekleme
X = np.append(arr=np.ones((14,1)).astype(int), values=sonveriler.iloc[:,:-1], axis=1)
print(X)

[[ 1.  0.  0.  0.  0.  1. 85.]
 [ 1.  1.  0.  0.  0.  1. 80.]
 [ 1.  0.  1.  1.  0.  0. 83.]
 [ 1.  0.  1.  0.  1.  0. 70.]
 [ 1.  0.  1.  0.  1.  0. 68.]
 [ 1.  1.  0.  0.  1.  0. 65.]
 [ 1.  1.  1.  1.  0.  0. 64.]
 [ 1.  0.  0.  0.  0.  1. 72.]
 [ 1.  0.  1.  0.  0.  1. 69.]
 [ 1.  0.  1.  0.  1.  0. 75.]
 [ 1.  1.  1.  0.  0.  1. 75.]
 [ 1.  1.  1.  1.  0.  0. 72.]
 [ 1.  0.  1.  1.  0.  0. 81.]
 [ 1.  1.  0.  0.  1.  0. 71.]]


In [43]:
#her bir değişkenin etki değerini bulma Backward eliminiation
x_l = sonveriler.iloc[:,[0,1,2,3,4,5]].values
x_l = np.array(x_l, dtype=float)
model = sm.OLS(sonveriler.iloc[:,-1:],x_l).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               humidity   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                 -0.148
Method:                 Least Squares   F-statistic:                    0.6653
Date:                Sat, 02 Dec 2023   Prob (F-statistic):              0.661
Time:                        22:18:37   Log-Likelihood:                -49.542
No. Observations:                  14   AIC:                             111.1
Df Residuals:                       8   BIC:                             114.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -4.0286      7.229     -0.557      0.5

In [46]:
x_l = sonveriler.iloc[:,[1,2,3,4,5]].values
x_l = np.array(x_l, dtype=float)
model = sm.OLS(sonveriler.iloc[:,-1:],x_l).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               humidity   R-squared:                       0.266
Model:                            OLS   Adj. R-squared:                 -0.060
Method:                 Least Squares   F-statistic:                    0.8165
Date:                Sat, 02 Dec 2023   Prob (F-statistic):              0.546
Time:                        22:34:28   Log-Likelihood:                -49.809
No. Observations:                  14   AIC:                             109.6
Df Residuals:                       9   BIC:                             112.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -6.2865      6.909     -0.910      0.3

/home/huseyin/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [50]:
x_train = x_train.iloc[:,1:]
x_test = x_test.iloc[:,1:]
regressor.fit(x_train, y_train)
y_pred = regressor.predict(x_test)
print(y_pred)

[[86.03731343]
 [83.3681592 ]
 [82.90713101]
 [82.44610282]
 [81.17827529]]
